In [9]:
import cv2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


video1 =cv2.VideoCapture(0)
video2 =cv2.VideoCapture(1)

while True:
    ret0,frame0 = video1.read()
    ret1,frame1 = video2.read()

    if (ret0):
        cv2.imshow('Camera 0',frame0)

    if (ret1):
        cv2.imshow('Camera 1',frame1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video1.release()
video2.release()

cv2.destroyAllWindows()

In [3]:

import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import *


model=YOLO('yolov8s.pt')


def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB',RGB)

video1 =cv2.VideoCapture(0)
video2 =cv2.VideoCapture(1)
video3 = cv2.VideoCapture(2)
my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")
print(class_list)
count=0
tracker = Tracker()

car_entering = {}
count_car = set()
area = [(523,73),(216,477),(825,489),(994,145)]
while True:
    
    ret0,frame0 = video1.read()
    ret1,frame1 = video2.read()
    ret2,frame2 = video3.read()

    if not ret0:
        break
    count += 1
    if count % 3 != 0:
        continue
    frame0=cv2.resize(frame0,(1020,500))
    frame1=cv2.resize(frame1,(1020,500))
    frame2 = cv2.resize(frame2,(1020,500))
    

    results1=model.predict(frame0)
    results2 = model.predict(frame1)
    results3 = model.predict(frame2)

    a=results1[0].boxes.data
    b=results2[0].boxes.data
    c=results3[0].boxes.data
    px = pd.DataFrame(a).astype('float')
    py = pd.DataFrame(b).astype('float')
    pz = pd.DataFrame(c).astype('float')
    list1 = []
    list2 = []
    list3 = []
    for index,row in px.iterrows():
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        '''if 'car' in c:
            result = cv2.pointPolygonTest(np.array(area,np.int32),((x1,y2)),False)
            if result >= 0:
                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
                cv2.circle(frame,(x1,y2),4,(2555,0,0),-1)
                cv2.putText(frame,str(c),(x1,y1),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))'''
        if 'car' in c:
           list1.append([x1,y1,x2,y2])
    bbox_id = tracker.update(list1)
    for bbox in bbox_id:
        x3,y3,x4,y4,id = bbox
        result=cv2.pointPolygonTest(np.array(area,np.int32),((x4,y4)),False)
        if result >= 0:
            car_entering[id]=(x4,y4)
            cv2.rectangle(frame0,(x3,y3),(x4,y4),(0,255,0),2)
            cv2.circle(frame0,(x4,y4),5,(2555,0,0),-1)
            cv2.putText(frame0,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))
            count_car.add(id)
    #webcam2
    for index,row in py.iterrows():
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        '''if 'car' in c:
            result = cv2.pointPolygonTest(np.array(area,np.int32),((x1,y2)),False)
            if result >= 0:
                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
                cv2.circle(frame,(x1,y2),4,(2555,0,0),-1)
                cv2.putText(frame,str(c),(x1,y1),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))'''
        if 'car' in c:
           list2.append([x1,y1,x2,y2])
    cbox_id = tracker.update(list2)
    for bbox in cbox_id:
        x3,y3,x4,y4,id = bbox
        result=cv2.pointPolygonTest(np.array(area,np.int32),((x4,y4)),False)
        if result >= 0:
            car_entering[id]=(x4,y4)
            cv2.rectangle(frame1,(x3,y3),(x4,y4),(0,255,0),2)
            cv2.circle(frame1,(x4,y4),5,(2555,0,0),-1)
            cv2.putText(frame1,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))
            #count_car.add(id) remember to countcar
    #webcam3
    for index,row in pz.iterrows():
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        '''if 'car' in c:
            result = cv2.pointPolygonTest(np.array(area,np.int32),((x1,y2)),False)
            if result >= 0:
                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
                cv2.circle(frame,(x1,y2),4,(2555,0,0),-1)
                cv2.putText(frame,str(c),(x1,y1),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))'''
        if 'car' in c:
           list3.append([x1,y1,x2,y2])
    dbox_id = tracker.update(list3)
    for bbox in dbox_id:
        x3,y3,x4,y4,id = bbox
        result=cv2.pointPolygonTest(np.array(area,np.int32),((x4,y4)),False)
        if result >= 0:
            car_entering[id]=(x4,y4)
            cv2.rectangle(frame2,(x3,y3),(x4,y4),(0,255,0),2)
            cv2.circle(frame2,(x4,y4),5,(2555,0,0),-1)
            cv2.putText(frame2,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))
            #count_car.add(id) remember to countcar

        
    cv2.polylines(frame0,[np.array(area,np.int32)],True,(255,0,255),2) 
    cv2.polylines(frame1,[np.array(area,np.int32)],True,(255,0,255),2)
    cv2.polylines(frame2,[np.array(area,np.int32)],True,(255,0,255),2)
    print(len(count_car))    

    if (ret0):
        cv2.imshow('Camera 0',frame0)

    if (ret1):
        cv2.imshow('Camera 1',frame1)

    if (ret2):
        cv2.imshow('Camera 2',frame2)

    if cv2.waitKey(1)&0xFF== ord('q'):
        break


video1.release()
video2.release()
video3.release()
cv2.destroyAllWindows()

number_of_cars = len(count_car)


































['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

0: 320x640 (no detections), 451.0ms
Speed: 8.5ms preprocess, 451.0ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0:

In [9]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import *


model=YOLO('yolov8s.pt')


def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB',RGB)

cap=cv2.VideoCapture(0)

my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")
print(class_list)
count=0
tracker = Tracker()

car_entering = {}
count_car = set()
area = [(815,380),(700,55),(612,71),(701,396)]
while True:
    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))
    

    results=model.predict(frame)
    a=results[0].boxes.data
    px = pd.DataFrame(a).astype('float')
    list = []
    for index,row in px.iterrows():
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        '''if 'car' in c:
            result = cv2.pointPolygonTest(np.array(area,np.int32),((x1,y2)),False)
            if result >= 0:
                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
                cv2.circle(frame,(x1,y2),4,(2555,0,0),-1)
                cv2.putText(frame,str(c),(x1,y1),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))'''
        if 'car' in c:
           list.append([x1,y1,x2,y2])
    bbox_id = tracker.update(list)
    for bbox in bbox_id:
        x3,y3,x4,y4,id = bbox
        result=cv2.pointPolygonTest(np.array(area,np.int32),((x4,y4)),False)
        if result >= 0:
            car_entering[id]=(x4,y4)
            cv2.rectangle(frame,(x3,y3),(x4,y4),(0,255,0),2)
            cv2.circle(frame,(x4,y4),5,(2555,0,0),-1)
            cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))
            count_car.add(id)
        
    cv2.polylines(frame,[np.array(area,np.int32)],True,(255,0,255),2) 
    print(len(count_car))    
    cv2.imshow("RGB", frame)
    if cv2.waitKey(1)&0xFF== ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

0: 320x640 (no detections), 368.1ms
Speed: 15.6ms preprocess, 368.1ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
0


In [10]:
#signal counter
flow_rate = number_of_cars/1

flow_rate1 = number_of_cars/0.5

approach_width = 4

saturation_flow = 525*approach_width

Y_ratio = saturation_flow/flow_rate

y_ratio_2 = saturation_flow/flow_rate1

critical_Y_ratio = Y_ratio + y_ratio_2

number_of_phases = 2

time_loss = 2

all_red = 4

loss_time_per_cycle = number_of_phases*time_loss + all_red


Total_cycle_length = (1.5*loss_time_per_cycle + 5)/(1-critical_Y_ratio)

green_signal_nb  = round(abs( Y_ratio/critical_Y_ratio *(Total_cycle_length-loss_time_per_cycle)))

print(green_signal_nb)


ZeroDivisionError: float division by zero

In [11]:
%pip install pyserial

   ---------------------------------------- 0.0/90.6 kB ? eta -:--:--
   ------------- -------------------------- 30.7/90.6 kB ? eta -:--:--
   ------------- -------------------------- 30.7/90.6 kB ? eta -:--:--
   ------------- -------------------------- 30.7/90.6 kB ? eta -:--:--
   ------------- -------------------------- 30.7/90.6 kB ? eta -:--:--
   ------------- -------------------------- 30.7/90.6 kB ? eta -:--:--
   ------------- -------------------------- 30.7/90.6 kB ? eta -:--:--
   ------------------------------------ --- 81.9/90.6 kB 255.1 kB/s eta 0:00:01
   ---------------------------------------- 90.6/90.6 kB 233.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
